In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
np.random.seed(42)
from PIL import Image # to handle images
from scipy.ndimage.filters import gaussian_filter
from scipy.stats import ttest_ind
from keras.models import Sequential
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPool2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


## import labels

In [27]:
#dflab = pd.read_json("D:/paredolia/noFacesInTheWild/data/annotations/captions_val2017.json", orient='split')
import json

with open('D:/paredolia/noFacesInTheWild/data/annotations/instances_val2017.json') as json_data:
    data = json.load(json_data)

In [28]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [29]:
pd.DataFrame.from_dict(data["categories"])


,supercategory,id,name
0,person,1,person
1,vehicle,2,bicycle
2,vehicle,3,car
3,vehicle,4,motorcycle
4,vehicle,5,airplane
...,...,...,...
75,indoor,86,vase
76,indoor,87,scissors
77,indoor,88,teddy bear
78,indoor,89,hair drier


In [30]:
pd.DataFrame.from_dict(data["annotations"])


,segmentation,area,iscrowd,image_id,bbox,category_id,id
0,"[[510.66, 423.01, 511.72, 420.03, 510.45, 416....",702.10575,0,289343,"[473.07, 395.93, 38.65, 28.67]",18,1768
1,"[[289.74, 443.39, 302.29, 445.32, 308.09, 427....",27718.47630,0,61471,"[272.1, 200.23, 151.97, 279.77]",18,1773
2,"[[147.76, 396.11, 158.48, 355.91, 153.12, 347....",78969.31690,0,472375,"[124.71, 196.18, 372.85, 356.81]",18,2551
3,"[[260.4, 231.26, 215.06, 274.01, 194.33, 307.6...",108316.66515,0,520301,"[112.71, 154.82, 367.29, 479.35]",18,3186
4,"[[200.61, 253.97, 273.19, 318.49, 302.43, 336....",75864.53530,0,579321,"[200.61, 89.65, 400.22, 251.02]",18,3419
...,...,...,...,...,...,...,...
36776,"{'counts': [94823, 6, 473, 8, 471, 10, 469, 11...",3773.00000,1,15517,"[197, 248, 264, 45]",6,900600015517
36777,"{'counts': [277, 2, 361, 9, 1, 17, 3, 17, 3, 8...",112181.00000,1,439994,"[0, 0, 427, 458]",1,900100439994
36778,"{'counts': [2770, 6, 418, 8, 416, 10, 86, 6, 3...",47024.00000,1,117719,"[6, 75, 474, 263]",44,904400117719
36779,"{'counts': [3912, 10, 363, 18, 356, 23, 301, 1...",27277.00000,1,50149,"[10, 41, 403, 152]",52,905200050149


In [25]:
data["info"]

{'description': 'COCO 2017 Dataset',
 'url': 'http://cocodataset.org',
 'version': '1.0',
 'year': 2017,
 'contributor': 'COCO Consortium',
 'date_created': '2017/09/01'}